# Dataset creation

## Installation

In [101]:
!pip install -q llama-cpp-python textdescriptives argilla==1.18 transformers datasets langdetect langchain


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [102]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 41.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-md
    Found existing installation: en-core-web-md 3.7.0
    Uninstalling en-core-web-md-3.7.0:
      Successfully uninstalled en-core-web-md-3.7.0

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [103]:
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 870.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.18-cp39-cp39-macosx_13_0_x86_64.whl size=1886255 sha256=635b308f295db89a312328747563c1229ae652af1940ec3a3cbd798cfd6e9562
  Stored in directory: /private/var/folders/9t/msy700h16jz3q35qvg4z1ln40000gn/T/pip-ephem-wheel-cache-y4tj1_uj/wheels/f5/c3/64/dceaadaf16197204d2344811b0c2b684c420f545bc0c27fad9
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensio

In [ ]:
# from ctransformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral")

# print(llm("AI is going to"))

## Imports

In [104]:
import textdescriptives as td
from datasets import load_dataset
import re
import spacy
from langdetect import detect
import argilla as rg
import numpy as np
import concurrent.futures
import requests
import json

/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to Argilla

In [2]:
import os
import argilla as rg

rg.init(api_url=os.environ.get("ARGILLA_API_URL_PRE"), api_key=os.environ.get("ARGILLA_API_KEY_PRE"))

/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.19.0 using a different client version (1.18.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


## Pre-processing

End-to-end workflow to create a dataset in Argilla with text measurements as metadata.
This aids in quickly identifying and improving potential dataset issues.

### Dataset creation

At first, we need to create a dataset in Argilla. This can either be done by loading a previous created dataset or by creating a new one. In order to avoid duplication, we will check if the dataset already exists. Additionally, we will load the markdown file that contains the dataset guidelines.


In [3]:
with open("GUIDELINES.md") as f:
    guidelines = f.read()
guidelines

"# Guidelines\n\nThe ShareGPT dataset is a dataset that was collected by public users who were using the Google Chrome extension offered by [sharegpt.com](sharegpt.com) to share their ChatGPT conversations. This data should mimic real-life usage of the model and can therefore be used to fine-tune a model for an actual scenario. Additionally, Google was accused of using this dataset as a baseline to train its [BARD](https://www.theverge.com/2023/3/29/23662621/google-bard-chatgpt-sharegpt-training-denies) model.\n\nWe decided to use a random subset of the raw version of the dataset including all conversations but we did filter out non-English conversation. The raw dataset used can be found on [the Hugging Face hub](https://huggingface.co/datasets/zetavg/ShareGPT-Processed).\n\n## Classification Tasks\n\nThe dataset aims to classify three things:\n\n1. Quality\n2. Intent\n3. Toxicity\n\n### Quality\n\nFor the quality, we have decided to define a rating question on a scale from 1 to 7. Thi

In [4]:
try:
    ds_local = rg.FeedbackDataset(
        fields=[
            rg.TextField(name="prompt", title="Prompt", use_markdown=True),
            rg.TextField(name="response", title="Response", use_markdown=True),
        ],
        questions=[
            rg.RatingQuestion(
                name="prompt-quality", 
                title="Prompt Quality",
                values=list(range(1, 8)), 
                description="How would you rate the quality of the prompt?",
            ),
            rg.LabelQuestion(
                name="prompt-intent", 
                title="Prompt Intent",
                labels=["generation", "rewrite", "extract", "closed-qa", "open-qa", "classification", "summarization", "brainstorming", "chat", "code", "other"], 
                description="What is the intent of the prompt?"
            ),
            rg.MultiLabelQuestion(
                name="response-toxicity", 
                title="Response Toxicity",
                labels=["illegal", "harmfull", "unqualified advice"], 
                description="What are the toxicities in the response (if any)?",
                required=False
            )
        ]
    )
    ds_remote = ds_local.push_to_argilla("sharegpt")
except Exception as e:
    ds_remote = rg.FeedbackDataset.from_argilla("sharegpt")
ds_remote

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:29                                                                                   │
│                                                                                                  │
│   26 │   │   │   )                                                                               │
│   27 │   │   ]                                                                                   │
│   28 │   )                                                                                       │
│ ❱ 29 │   ds_remote = ds_local.push_to_argilla("sharegpt")                                        │
│   30 except Exception as e:                                                                      │
│   31 │   ds_remote = rg.FeedbackDataset.from_argilla("sharegpt")                                 │
│   32 ds_remote                                                                                   │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/feedback/dataset/local/mixins.py:256 in push_to_argilla            │
│                                                                                                  │
│   253 │   │   httpx_client: "httpx.Client" = client.http_client.httpx                            │
│   254 │   │                                                                                      │
│   255 │   │   if workspace is None:                                                              │
│ ❱ 256 │   │   │   workspace = Workspace.from_name(client.get_workspace())                        │
│   257 │   │                                                                                      │
│   258 │   │   if isinstance(workspace, str):                                                     │
│   259 │   │   │   workspace = Workspace.from_name(workspace)                                     │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/workspaces.py:367 in from_name                                     │
│                                                                                                  │
│   364 │   │   │   if ws.name == name:                                                            │
│   365 │   │   │   │   return cls._new_instance(client, ws)                                       │
│   366 │   │                                                                                      │
│ ❱ 367 │   │   raise ValueError(                                                                  │
│   368 │   │   │   f"Workspace with name=`{name}` doesn't exist in Argilla, so please"            │
│   369 │   │   │   " create it via the `Workspace.create` method as follows:"                     │
│   370 │   │   │   f" `Workspace.create('{name}')`."                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Workspace with name=`None` doesn't exist in Argilla, so please create it via the `Workspace.create` 
method as follows: `Workspace.create('None')`.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:31                                                                                   │
│                                                                                                  │
│   28 │   )                                                                                       │
│   29 │   ds_remote = ds_local.p

#### Configure the metadata-properties

Next we will be using `text-descriptives` to configure the metadata-properties. This will be used to add and updat relevant metadata-properties to the dataset. Because `text-descriptives` doesn't provide any programmatic interface with the metrics-groups and their sub-metrics, we will run the computation on the an example text and use the results to configure the metadata-properties.

In [116]:
metric_group = ["descriptive_stats"]
relevant_subgroups = []
df_metrics = td.extract_metrics(
    text=["this is an example prompt"], 
    lang="en", 
    metrics=metric_group,
    spacy_model="en_core_web_sm"
).drop(columns=["text"] + relevant_subgroups if relevant_subgroups else ["text"])
df_metrics.columns

ℹ Both a spacy model and a language were provided. Will use the spacy
model and ignore language.


Index(['token_length_mean', 'token_length_median', 'token_length_std',
       'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
       'syllables_per_token_mean', 'syllables_per_token_median',
       'syllables_per_token_std', 'n_tokens', 'n_unique_tokens',
       'proportion_unique_tokens', 'n_characters', 'n_sentences'],
      dtype='object')

Next, we will be working on converting the `text-descriptives` output to a format that can be used to configure the metadata-properties for our supported types: `TermsMetadataProperty`, `IntegerMetadataProperty` and `FloatMetadataProperty`. Note that we are also applying some subjective formatting choices to ensure that the metadata-properties are easy to read and understand.

In [113]:
def clean_column_name(col_name):
    """Clean a column name to fit a specific regex pattern."""
    col_name = col_name.lower()  # Convert to lowercase
    col_name = re.sub(r'[^a-z0-9_]', '_', col_name)  # Replace non-alphanumeric characters with underscores
    return col_name

def create_metadata_properties(df, prefix):
    """Generate metadata properties based on dataframe columns and data types."""
    properties = []
    for col, dtype in df.dtypes.items():
        name = f"{prefix}_{clean_column_name(col)}"
        title = name.replace('_', ' ').title()

        if dtype == 'object':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        elif dtype == 'int64':
            prop = rg.IntegerMetadataProperty(name=name, title=title)
        elif dtype == 'float64':
            prop = rg.FloatMetadataProperty(name=name, title=title)
        elif dtype == 'bool':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        else:
            print(f"Unhandled data type for column {col}: {dtype}")
            continue
        properties.append(prop)
    return properties

metadata_properties = []
metadata_properties += create_metadata_properties(df_metrics, 'prompt')
metadata_properties += create_metadata_properties(df_metrics, 'response')
for metadata_property in metadata_properties:
    try:
        field = ds_remote.metadata_property_by_name(metadata_property.name)
        if not field:
            ds_remote.add_metadata_property(metadata_property)
    except (KeyError, ValueError) as e:
        ds_remote.add_metadata_property(metadata_property)        
ds_remote.metadata_properties

[RemoteFloatMetadataProperty(id=UUID('9691b808-c85e-4248-98bd-89bc2a81b2eb'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_mean', title='Response Token Length Mean', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('949d3bf5-a8ec-47c8-a4ce-249a9ee4a922'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_median', title='Response Token Length Median', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('43c7f5a7-e856-4794-9da8-6f89848a3a01'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_std', title='Response Token Length Std', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('4dcca678-229c-43e8-b0d0-9f5fee2ec902'), client=<httpx.Client object at 0x11105d3d0>, name='response_sentence_length_mean', title='Response Sentence Length Mean', visible_for_annotators=T

### Data collection

In [120]:
dataset = load_dataset("zetavg/ShareGPT-Processed")
dataset = dataset["train"]
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'lang'],
        num_rows: 90665
    })
})

In [130]:
dataset = dataset.filter(function=lambda x: x.get("lang", "?") == "en")
dataset = dataset.filter(lambda x: x["conversations"][0]["from"] == "human")
dataset = dataset.filter(lambda x: len(x["conversations"])>1)
dataset

Filter: 100%|██████████| 9962/9962 [00:02<00:00, 4883.42 examples/s]


Dataset({
    features: ['id', 'conversations', 'lang'],
    num_rows: 9772
})

In [134]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(list(range(min(10000, len(dataset)))))
dataset

Dataset({
    features: ['id', 'conversations', 'lang'],
    num_rows: 9772
})

In [135]:
dataset = dataset.map(lambda x: {"prompt": x["conversations"][0]["value"], "response": x["conversations"][1]["value"]})
dataset

Map: 100%|██████████| 9772/9772 [00:03<00:00, 2621.48 examples/s]


Dataset({
    features: ['id', 'conversations', 'lang', 'prompt', 'response'],
    num_rows: 9772
})

In [136]:
# Extract metrics
spacy_model = "en_core_web_md" # we need a model with vectors
df_prompt = td.extract_metrics(text=dataset["prompt"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])
df_response = td.extract_metrics(text=dataset["response"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])

# Identify integer and boolean columns for prompts and responses
int_cols_prompts = df_prompt.select_dtypes(include=['int64']).columns.tolist()
bool_cols_prompts = df_prompt.select_dtypes(include=['boolean']).columns.tolist()

int_cols_responses = df_response.select_dtypes(include=['int64']).columns.tolist()
bool_cols_responses = df_response.select_dtypes(include=['boolean']).columns.tolist()

# Combine column lists for prompts and responses
int_cols = list(set(int_cols_prompts + int_cols_responses))
bool_cols = list(set(bool_cols_prompts + bool_cols_responses))
int_cols, bool_cols

(['n_tokens', 'n_unique_tokens', 'n_characters', 'n_sentences'], [])

Next, we will be casting the `numpy`-datatypes to basic Python built-in datatypes. This is required because the Argilla client doesn't support `numpy`-datatypes.

In [137]:
# --- Functions ---
def cast_to_python_types(df):
    """
    Convert integer and boolean columns to Python native types.
    """
    for column in df.columns:
        df[column].fillna(0, inplace=True)
        if df[column].dtype == bool:
            df[column] = df[column].astype(str)
        elif df[column].dtype == np.int64:
            df[column] = df[column].astype(int)
        elif df[column].dtype == np.float64:
            df[column] = df[column].astype(float)
        else:
            print(f"Unhandled data type for column {column}: {df[column].dtype}")
    return df

df_prompt = cast_to_python_types(df_prompt)
df_response = cast_to_python_types(df_response)

Lastly, we will loop through the Hugging Face dataset, add the metadata-properties and update the Argilla dataset with the new records.

In [138]:
# Prepare feedback records with metadata and suggestions
records = []

cols_with_values_other_than_zeros_or_nan_prompt = df_prompt.columns[~(df_prompt.fillna(0) == 0).all() & ~df_prompt.isnull().any()].tolist()
cols_with_values_other_than_zeros_or_nan_response = df_response.columns[~(df_response.fillna(0) == 0).all() & ~df_response.isnull().any()].tolist()


for i, record in enumerate(dataset):
    # Prepare metadata for prompts
    metadata_prompts = {f"prompt_{col}": value for col, value in df_prompt[cols_with_values_other_than_zeros_or_nan_prompt].iloc[i].items()}
    # Prepare metadata for responses
    metadata_response = {f"response_{col}": value for col, value in df_response[cols_with_values_other_than_zeros_or_nan_response].iloc[i].items()}

    # Explicitly cast integers using Python's native int type
    for col in int_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = int(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = int(metadata_response[f"response_{col}"])

    # Convert booleans to strings using Python's native str type
    for col in bool_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = str(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = str(metadata_response[f"response_{col}"])

    # Combine both metadata dictionaries into one
    metadata = {**metadata_prompts, **metadata_response}
    record = rg.FeedbackRecord(
        fields={"prompt": record["prompt"], "response": record["response"]},
        metadata=metadata,
    )
    records.append(record)

# Add records to the dataset and push to Argilla
ds_remote.add_records(records)

Pushing records to Argilla...: 100%|██████████| 306/306 [00:41<00:00,  7.41it/s]
